In [1]:
import pandas as pd
import os
from collections import Counter
import glob
from tqdm.notebook import tqdm
from gensim.models import word2vec
import collections
from scipy.stats import entropy
import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from itertools import combinations
import umap
import pacmap
import numpy as np
from scipy.spatial import ConvexHull
from numpy.linalg import norm
from numpy import dot
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import warnings
import re
import json
import csv

In [2]:
model=word2vec.Word2Vec.load('../Data/Models/word2vec_epoch_1000_min_1.model')

In [3]:
f = open('../Data/data.json')
data = json.load(f)

a = open('../Data/added_data.json')
added_data = json.load(a)

In [4]:
X = model.wv.vectors

In [5]:
with open('../Data/sequences.csv', 'r') as f:
    reader = csv.reader(f)
    sequences = list(reader)

In [6]:
# tsne
def tsne_plot(X):
    X=np.array(X)
    #pca = PCA(n_components=50)
    #X_pca=pca.fit_transform(X)
    proj = TSNE().fit_transform(X)
    return proj
proj=tsne_plot(X)

# UMAP 
proj_1 = umap.UMAP(n_components=2, metric='cosine').fit_transform(X)

# pca 
pca = PCA(n_components=2)
proj_2=pca.fit_transform(X)

In [7]:
# pacmap 
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0) 
proj_3 = embedding.fit_transform(X, init="pca")


get list of unique zipcodes

In [8]:
zips = {}

In [9]:
import re
#retrieve zipcode
def getZip(info):
    pattern = '\d{4}'
    match = re.search(pattern, info)
    if match is not None:
        return match.group()
    else :
        return

def add_to_dict(zip_code):
    if str.startswith(zip_code,'1'):
        zip_code = '1000'
    #check if it is already in the dictionary
    if zip_code in zips:
        zips[zip_code] += 1
    else:
        zips[zip_code] = 1
    return zip_code

# # Extract zip codes 
address_zips = {}
wrong_addresses = []
# we have some json objects where the address exists but the info is null. This happens in 1602 cases (assume this is where the geolocator failed)
for address, info in data.items():
    if info is not None: 
        zip_code = getZip(info['address'])  # Extract zip code from address could be empty
        if zip_code is not None:
            if 1000 <= int(zip_code) < 4999: #for now removing all failed zipcodes in Jylland
                #get list of unique zipcodes
                zip = add_to_dict(zip_code)
                address_zips[address] = zip
            else: 
                wrong_addresses.append(address) #go through later
        else:
            address_zips[address] = 0000 #no zipcode was found
    else:
        address_zips[address] = 0000 #no info was found on the address?

#run through the list of wrong addresses and try to find them in the added_data.json file to get the "correct" zipcode
for address in wrong_addresses:
    #find in json file
    if address in added_data:
        zip = getZip(added_data[address]['address'])
        address_zips[address] = zip

for address, zip_code in address_zips.items():
    print(address)
    print(zip_code)
    print()

print(len(address_zips))

Otto Mallings Gade 8
2100

Borgbjergsvej 56
2450

Nørre Farimagsgade 64A
1000

Nyrnberggade 38
2300

Dag Hammarskjölds Allé 32
2100

Vigerslev Allé 71
2500

Arkonagade 19
1000

Lyongade 9
2300

Bachersmindevej 5B
2791

Thingvalla Allé 25
2300

Ungarnsgade 4
2300

Maglestien 20
2860

Bag Elefanterne 15
1000

Alsgade 8
1000

Tomsgårdsvej 23C
2400

Wilkensvej 8
2000

Tåsingegade 20
2100

Korsgade 38
2200

Belfastvej 13B
2300

Alliancevej
2450

Kyringevej 1
2700

Hvidkildevej 36B
2400

Gammel Køge Landevej 132
2500

Nørrebrogade 210
2200

Amagerbrogade 289
2300

Leiden Alle 15
2791

Blegdamsvej 132A
2100

Langelandsvej 10A
2000

Burmeistersgade 1C
1000

Bryggernes Plads 9A
1000

Scandiagade 58
2450

Halmtorvet 29E
1000

Helsinkigade 5
2150

Gåsebæksvej 12
2500

Nadine Gordimers Vej 11
2300

Baggesensgade 39
2200

Pilegård Alle 3
2770

I C Lembrechts Alle 13
2650

Ingrid Marievej 1
2500

Tilburg Alle 46
2791

Kastrupvej 102
4370

Raffinaderivej 2300 København S
0

Sundby St.
0

Lindevej 25


In [10]:
print(zips)

{'2100': 2759, '2450': 1419, '1000': 7143, '2300': 6318, '2500': 2099, '2791': 774, '2860': 206, '2400': 1867, '2000': 1776, '2200': 2698, '2700': 1452, '2150': 302, '2770': 2051, '2650': 299, '4370': 28, '4780': 19, '2900': 525, '3700': 44, '2610': 225, '2720': 778, '3770': 11, '3390': 48, '4760': 24, '4930': 17, '3000': 21, '3730': 25, '4800': 79, '4400': 23, '4583': 17, '4850': 7, '2750': 17, '3630': 25, '4990': 17, '2670': 12, '4960': 3, '2870': 109, '3330': 12, '3500': 22, '4000': 59, '4840': 10, '4040': 24, '2765': 3, '3200': 5, '3310': 13, '4532': 1, '2600': 25, '3650': 5, '4160': 11, '3400': 33, '4720': 14, '3520': 10, '3080': 10, '4621': 2, '4200': 109, '2830': 19, '3060': 15, '2620': 22, '4100': 18, '3460': 20, '3070': 16, '4520': 2, '4653': 5, '3600': 19, '2850': 6, '3100': 9, '3790': 5, '4330': 18, '2605': 19, '3250': 50, '3660': 6, '2730': 27, '2942': 4, '4480': 10, '4571': 9, '4581': 13, '4690': 22, '3720': 8, '4622': 6, '3670': 9, '4970': 13, '4270': 8, '2635': 10, '2690

In [11]:
import matplotlib.pyplot as plt

# Define color palette for zip codes
zip_colors = {}  # Dictionary to map zip codes to colors
unique_zip_codes = set(address_zips.values())
num_colors = len(unique_zip_codes)
color_palette = plt.cm.get_cmap('tab20', num_colors)  # Choose a colormap

for idx, zip_code in enumerate(unique_zip_codes):
    zip_colors[zip_code] = color_palette(idx)

# Create the plots
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
xs = [proj_2, proj, proj_1, proj_3]
ts = ["PCA", "t-SNE", "UMAP", "PaCMAP"]
s = 0

print(ax)

# for row in ax:
#     for col in row:
#         print("new plot")
#         col.set_title(ts[s], fontsize=25)
#         for zip_code in unique_zip_codes:
#             # Filter data based on zip code
#             journeys_with_zip = [journey for journey, zip_code_journey in address_zips.items() if zip_code_journey == zip_code]
#             col.scatter(
#                 [xs[s][idx][0] for idx, journey in enumerate(proj) if any(j in journeys_with_zip for j in journey)],
#                 [xs[s][idx][1] for idx, journey in enumerate(proj) if any(j in journeys_with_zip for j in journey)],
#                 lw=0.1, s=5, edgecolors='white',
#                 label=zip_code, c=zip_colors[zip_code]
#             )
#         s += 1
#         col.axis('off')


# # Create a custom legend
# handles = [plt.Line2D([0], [0], marker='o', color='w', markersize=10, markerfacecolor=zip_colors[zip_code], label=zip_code) for zip_code in unique_zip_codes]
# plt.legend(handles=handles, title='Zip Code')

# fig.tight_layout()
# plt.show()


new plot


/var/folders/z3/0j97vknj4696wz9q6_jg32240000gn/T/ipykernel_10802/1007913705.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_palette = plt.cm.get_cmap('tab20', num_colors)  # Choose a colormap


: 